<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/07Jul2022_MLB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/mlb_batting.zip -O mlb.zip

In [59]:
import shutil
from pathlib import Path
import pandas as pd

In [60]:
shutil.unpack_archive('mlb.zip', extract_dir='mlb')

In [61]:
files=list(Path('mlb').glob('**/*.csv'))
keys=[afile.as_posix().split('/')[1:] for afile in files]
keys=[tuple((key[0], key[1], key[-1].split('.')[0])) for key in keys]

In [62]:
df_list=[]
days=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for afile in files:
    df=pd.read_csv(afile).iloc[:,1:]
    df['home']=df['OPP'].str.startswith('vs').astype(int)
    df['win']=(df['Result'].str[0]=='W').astype(int)
    df=df.loc[df['Date'].str[:3].isin(days),['Date','AB','HR','home','win']]
    df[['day','Date']]=df.loc[:,'Date'].str.split(' ',expand=True)
    df_list.append(df)

In [63]:
df=pd.concat(df_list,keys=keys)
df.index=df.index.droplevel(3)
df.index.names=['team','player','year']
df=df.reset_index().set_index(['team','player'])
df['date']=pd.to_datetime(df['Date'] + '/' + df['year'])
df=df.drop(['year','Date'],axis=1)

In [64]:
df

AB HR  home  win  day       date
team                player                                          
los-angeles-dodgers Freddie Freeman  4  0     1    0  Wed 2019-10-09
                    Freddie Freeman  5  0     0    0  Mon 2019-10-07
                    Freddie Freeman  4  0     0    1  Sun 2019-10-06
                    Freddie Freeman  3  0     1    1  Fri 2019-10-04
                    Freddie Freeman  4  1     1    0  Thu 2019-10-03
...                                 .. ..   ...  ...  ...        ...
pittsburgh-pirates  Oneil Cruz       5  0     1    0  Wed 2022-06-22
                    Oneil Cruz       4  0     1    1  Tue 2022-06-21
                    Oneil Cruz       5  0     1    1  Mon 2022-06-20
                    Oneil Cruz       4  1     1    0  Sun 2021-10-03
                    Oneil Cruz       5  0     1    1  Sat 2021-10-02

[252919 rows x 6 columns]

In [65]:
df[['AB','HR']]=df[['AB','HR']].astype(int)

In [66]:
df

AB  HR  home  win  day       date
team                player                                            
los-angeles-dodgers Freddie Freeman   4   0     1    0  Wed 2019-10-09
                    Freddie Freeman   5   0     0    0  Mon 2019-10-07
                    Freddie Freeman   4   0     0    1  Sun 2019-10-06
                    Freddie Freeman   3   0     1    1  Fri 2019-10-04
                    Freddie Freeman   4   1     1    0  Thu 2019-10-03
...                                  ..  ..   ...  ...  ...        ...
pittsburgh-pirates  Oneil Cruz        5   0     1    0  Wed 2022-06-22
                    Oneil Cruz        4   0     1    1  Tue 2022-06-21
                    Oneil Cruz        5   0     1    1  Mon 2022-06-20
                    Oneil Cruz        4   1     1    0  Sun 2021-10-03
                    Oneil Cruz        5   0     1    1  Sat 2021-10-02

[252919 rows x 6 columns]

In [67]:
team_HR_ratio=df.pivot_table(index=['team'],values=['AB','HR'],columns='home',aggfunc='sum')
team_HR_ratio=team_HR_ratio.rename(columns={0:"Away",1:"Home"},level=1)
team_HomeAway=team_HR_ratio['HR']/team_HR_ratio['AB']
team_HomeAway['ratio']=team_HomeAway['Home']/team_HomeAway['Away']
team_HomeAway.sort_values('ratio',ascending=False)

home,Away,Home,ratio
team,,,
colorado-rockies,0.033055,0.042727,1.292609
chicago-white-sox,0.032907,0.039290,1.193999
baltimore-orioles,0.035093,0.041690,1.187971
san-diego-padres,0.035341,0.041202,1.165843
los-angeles-dodgers,0.038326,0.044121,1.151208
seattle-mariners,0.039068,0.044925,1.149908
boston-red-sox,0.035879,0.040978,1.142131
houston-astros,0.032262,0.036200,1.122068
cincinnati-reds,0.035382,0.039470,1.115548


In [71]:
player_HR_ratio=df.pivot_table(index=['player'],values=['AB','HR'],columns='home',aggfunc='sum')
player_HR_ratio=player_HR_ratio.rename(columns={0:"Away",1:"Home"},level=1)
player_HomeAway=player_HR_ratio['HR']/player_HR_ratio['AB']
player_HomeAway=player_HomeAway.query('Away>0 and Home>0')  #avoid division by zero

player_HomeAway['ratio']=player_HomeAway['Home']/player_HomeAway['Away']
player_HomeAway=player_HomeAway.join(player_HR_ratio['AB'],rsuffix='AB')
player_HomeAway=player_HomeAway.query('AwayAB + HomeAB>1000')  #only look at players with over 1000 AB
player_HomeAway.sort_values('ratio',ascending=False)

home,Away,Home,ratio,AwayAB,HomeAB
player,,,,,
Kevin Plawecki,0.009901,0.027634,2.791019,606,579
Ryan McMahon,0.021154,0.054366,2.570016,520,607
Luis Arraez,0.005059,0.012963,2.562346,593,540
Cedric Mullins,0.019727,0.048465,2.456816,659,619
Hanser Alberto,0.009677,0.020505,2.118822,620,634
...,...,...,...,...,...
Donovan Solano,0.016949,0.008046,0.474713,944,870
Harrison Bader,0.045849,0.020888,0.455578,807,766
Johan Camargo,0.043261,0.018663,0.431391,601,643
